<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [2]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [3]:
DATA_PATH = '/workspace/sgd_logreg_nn/notebooks'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [4]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [5]:
df = df.sample(False, 0.5)

In [6]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (

Keeping only first two categorical features for simplicity

In [7]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [8]:
df = df.fillna(0, subset=num_columns)

## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

In [9]:
df.take(1)

[Row(_c0=1, _c1=0, _c2=-1, _c3=0, _c4=0, _c5=1465, _c6=0, _c7=17, _c8=0, _c9=4, _c10=0, _c11=4, _c12=0, _c13=0, _c14='241546e0', _c15='38a947a1', _c16='fa673455', _c17='6a14f9b9', _c18='25c83c98', _c19='fe6b92e5', _c20='1c86e0eb', _c21='1f89b562', _c22='a73ee510', _c23='e7ba2569', _c24='755e4a50', _c25='208d9687', _c26='5978055e', _c27='07d13a8f', _c28='5182f694', _c29='f8b34416', _c30='e5ba7672', _c31='e5f8f18f', _c32=None, _c33=None, _c34='f3ddd519', _c35=None, _c36='32c7478e', _c37='b34f3128', _c38=None, _c39=None, id=12)]

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

idx_columns = [col + "_idx" for col in cat_columns]
encoded_columns = [col + "_encoded" for col in cat_columns]

str_indexers = [StringIndexer(inputCol=col, outputCol=col + "_idx", handleInvalid='keep') for col in cat_columns]
encoder = OneHotEncoderEstimator(inputCols=idx_columns, outputCols=encoded_columns)
assembler = VectorAssembler(inputCols=num_columns + encoded_columns, outputCol="features")

### Fit and save pipeline

In [11]:
pipeline = Pipeline(stages=str_indexers + [encoder, assembler])

fitted = pipeline.fit(df)

In [12]:
PIPELINE_PATH = os.path.join(DATA_PATH, 'pipeline')

fitted.write().overwrite().save(PIPELINE_PATH)

### Load fitted pipeline

In [13]:
from pyspark.ml import PipelineModel

pipeline = PipelineModel.load(PIPELINE_PATH)

### Transform dataset using pipeline

In [14]:
new_df = pipeline.transform(df)
new_df = new_df.select('id', 'features', '_c0')
new_df.take(1)

[Row(id=12, features=SparseVector(1950, {1: -1.0, 4: 1465.0, 6: 17.0, 8: 4.0, 10: 4.0, 25: 1.0, 1398: 1.0}), _c0=1)]

### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [15]:
from pyspark.sql import Window

def split_by_col(df, split_col, parts_fractions):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    
    parts = []
    
    window = Window().orderBy(split_col)
    df = df.withColumn('percent_rank', F.percent_rank().over(window))
    
    for idx, fraction in enumerate(parts_fractions):
        part_values = df.filter(
            (F.col('percent_rank') > sum(parts_fractions[:idx])) &
            (F.col('percent_rank') <= sum(parts_fractions[:idx + 1]))).drop('percent_rank')
        
        parts.append(part_values)
    
    return parts

train_df, val_df, test_df = split_by_col(new_df, 'id', [0.8, 0.1, 0.1])

In [16]:
N = df.count()
N

1832839

In [17]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7999993452780086, 0.10000005456016595, 0.10000005456016595)

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [18]:
%%time
from pyspark.ml.classification import LogisticRegression

label_column = '_c0'

lr_model = LogisticRegression(featuresCol='features', labelCol=label_column)
lr_model = lr_model.fit(train_df)

CPU times: user 26.8 ms, sys: 13.4 ms, total: 40.2 ms
Wall time: 2min 28s


In [19]:
lr_model.transform(val_df).first()

Row(id=455266594741, features=SparseVector(1950, {2: 58.0, 3: 20.0, 4: 2891.0, 5: 20.0, 6: 9.0, 7: 23.0, 8: 20.0, 10: 1.0, 12: 20.0, 39: 1.0, 1402: 1.0}), _c0=0, rawPrediction=DenseVector([1.8442, -1.8442]), probability=DenseVector([0.8634, 0.1366]), prediction=0.0)

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [20]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics

def rocauc(model, df):
    predicts = model.transform(df).rdd.map(lambda row: (float(row['probability'][1]), float(row[label_column])))
    return BinaryClassificationMetrics(predicts).areaUnderROC


def compute_loss(row):
    proba, label = row
    return label * np.log(proba) + (1 - label) * np.log(1 - proba)


def logloss(df):
    return df.map(lambda row: compute_loss(row)).mean()


def ne(model, df):
    predicts = model.transform(df).rdd.map(lambda row: (float(row['probability'][1]), float(row[label_column])))
    
    l1_loss = logloss(predicts)
    avg_label = predicts.map(lambda row: row[1]).mean()
    
    return l1_loss / compute_loss([avg_label, avg_label])

In [21]:
%%time
# my metrics

print(rocauc(lr_model, val_df))
print(ne(lr_model, val_df))
print(rocauc(lr_model, test_df))
print(ne(lr_model, test_df))

0.7030084715755153
0.9236270891570331
0.7000710053901746
0.9262272701785131
CPU times: user 97.6 ms, sys: 13.4 ms, total: 111 ms
Wall time: 2min 33s


In [22]:
rocauc(lr_model, val_df)

0.7030084715755153

In [23]:
ne(lr_model, val_df)

0.9236270891570331

In [24]:
rocauc(lr_model, test_df)

0.7000710053901746

In [25]:
ne(lr_model, test_df)

0.9262272701785131

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [26]:
def process_df(df, is_train=False):
    df = df.fillna(0, subset=num_columns)
    df = pipeline.transform(df)
    
    selected_features = ['id', 'features']
    if is_train:
        selected_features += [label_column]
    
    return df.select(selected_features)

In [27]:
TEST_PATH = os.path.join(DATA_PATH, 'test.csv')

submit_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)

In [28]:
processed_df = process_df(df, is_train=True)
submit_df = process_df(submit_df)

In [29]:
sampled = processed_df.sample(False, 0.5)
train, test = split_by_col(sampled, 'id', [0.9, 0.1])

In [30]:
%%time
from sklearn.model_selection import ParameterGrid

param_grid = {
    'elasticNetParam': [0, 0.25, 0.5, 0.75, 1], 
    'fitIntercept': [True, False],
    'maxIter': [10, 50, 100],
    'regParam': [0, 0.01, 0.1, 0.2],
}

max_auc = -1
saved_model = None
max_params = None

for params in ParameterGrid(param_grid):
    print(params)
    lr_model = LogisticRegression(featuresCol='features', labelCol=label_column, **params)
    lr_model = lr_model.fit(train)
    
    cur_auc = rocauc(lr_model, test)
    
    if max_auc <= cur_auc:
        max_auc = cur_auc
        saved_model = lr_model
        max_params = params

{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0, 'maxIter': 10}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0.01, 'maxIter': 10}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0.1, 'maxIter': 10}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0.2, 'maxIter': 10}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0, 'maxIter': 50}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0.01, 'maxIter': 50}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0.1, 'maxIter': 50}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0.2, 'maxIter': 50}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0, 'maxIter': 100}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0.01, 'maxIter': 100}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0.1, 'maxIter': 100}
{'elasticNetParam': 0, 'fitIntercept': True, 'regParam': 0.2, 'maxIter': 100}
{'elasticNetParam': 0, 'fitIntercept': False, 'regParam': 0, 'maxIter': 10}

{'elasticNetParam': 1, 'fitIntercept': True, 'regParam': 0, 'maxIter': 100}
{'elasticNetParam': 1, 'fitIntercept': True, 'regParam': 0.01, 'maxIter': 100}
{'elasticNetParam': 1, 'fitIntercept': True, 'regParam': 0.1, 'maxIter': 100}
{'elasticNetParam': 1, 'fitIntercept': True, 'regParam': 0.2, 'maxIter': 100}
{'elasticNetParam': 1, 'fitIntercept': False, 'regParam': 0, 'maxIter': 10}
{'elasticNetParam': 1, 'fitIntercept': False, 'regParam': 0.01, 'maxIter': 10}
{'elasticNetParam': 1, 'fitIntercept': False, 'regParam': 0.1, 'maxIter': 10}
{'elasticNetParam': 1, 'fitIntercept': False, 'regParam': 0.2, 'maxIter': 10}
{'elasticNetParam': 1, 'fitIntercept': False, 'regParam': 0, 'maxIter': 50}
{'elasticNetParam': 1, 'fitIntercept': False, 'regParam': 0.01, 'maxIter': 50}
{'elasticNetParam': 1, 'fitIntercept': False, 'regParam': 0.1, 'maxIter': 50}
{'elasticNetParam': 1, 'fitIntercept': False, 'regParam': 0.2, 'maxIter': 50}
{'elasticNetParam': 1, 'fitIntercept': False, 'regParam': 0, 'maxIt

In [31]:
print(max_params)
print(max_auc)

{'elasticNetParam': 1, 'fitIntercept': True, 'regParam': 0, 'maxIter': 10}
0.7026011473055717


In [35]:
lr_model = LogisticRegression(featuresCol='features', labelCol=label_column, **max_params)
lr_model = lr_model.fit(sampled)

saved_model = lr_model

In [36]:
%%time
predicted_submit = saved_model.transform(submit_df).select('id', 'probability').toPandas()
predicted_submit.head()

CPU times: user 20.7 s, sys: 211 ms, total: 20.9 s
Wall time: 23.5 s


,id,probability
0,566935904713,"[0.7438637807325406, 0.25613621926745944]"
1,566935904715,"[0.7106944340329149, 0.28930556596708507]"
2,566935904727,"[0.689059261600075, 0.310940738399925]"
3,566935904737,"[0.48403990617722636, 0.5159600938227736]"
4,566935904741,"[0.711774128524775, 0.2882258714752251]"


In [37]:
predicted_submit['probability'] = predicted_submit['probability'] \
            .map(lambda probas: probas[-1] if not np.isnan(probas[-1]) else 0)

predicted_submit = predicted_submit.rename(columns={'probability': 'proba'})

In [38]:
SUBMIT_PATH = os.path.join(DATA_PATH, 'submit.csv')
predicted_submit.to_csv(SUBMIT_PATH, index=False)